In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install matplotlib numpy Pillow tensorflow opencv-python mediapipe

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import math
import PIL
import cv2
import time
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import image as keras_image

###**Model**

In [ ]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet34(shape = (180, 180, 3), classes = 2):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [ ]:
model = ResNet34()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.load_weights('/content/drive/MyDrive/rope_skipping/runs/2000-epochs/weights/best.h5')

###**MediaPipe**

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/rope_skipping/test videos/1.mp4')

In [ ]:
output_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('/content/drive/MyDrive/rope_skipping/test videos/outputs/16.mp4', cv2.VideoWriter_fourcc(*'mp4v'), output_fps, (output_width, output_height))

In [ ]:
def extract_keypoints(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    else:
        return None

In [ ]:
def prediction(frame):
    # Convert the frame to an image array
    img_array = keras_image.array_to_img(frame)  # Use the 'image' submodule from Keras
    img_array = img_array.resize((180, 180))  # Resize to match the target size
    img_array = keras_image.img_to_array(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Perform prediction
    predictions = model.predict(img_array)
    predicted_class = 1 if predictions[0][0] >= 0.5 else 0
    class_label = 'Skipping' if predicted_class == 1 else 'Not Skipping'
    return class_label

In [ ]:
font_scale_percentage = 0.001
font_scale = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) * font_scale_percentage

def write_text(frame, text, org=(50,50)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    #org = (50, 50)
    fontScale = font_scale
    color = (0, 0, 255)
    thickness = 2
    cv2.putText(frame, text, org, font, fontScale, color, thickness, cv2.LINE_AA)

In [ ]:
specific_keypoints_indices = [23, 24, 25, 26, 27, 28, 31, 32, 13, 14]

In [ ]:
def calculate_angle(a, b, c):
    # Extract X and Y coordinates of the keypoints
    x1, y1, _ = a  # Assuming a is a tuple (x, y, z)
    x2, y2, _ = b
    x3, y3, _ = c

    # Calculate vectors AB and BC
    vec_ab = np.array([x1 - x2, y1 - y2])
    vec_bc = np.array([x3 - x2, y3 - y2])

    # Calculate the dot product of AB and BC
    dot_product = np.dot(vec_ab, vec_bc)

    # Calculate the magnitudes of vectors AB and BC
    mag_ab = np.linalg.norm(vec_ab)
    mag_bc = np.linalg.norm(vec_bc)

    # Calculate the angle in radians
    radians = np.arccos(dot_product / (mag_ab * mag_bc))

    # Convert radians to degrees
    angle_deg = np.degrees(radians)

    # Ensure angle is between 0 and 180 degrees
    angle = angle_deg if angle_deg <= 180.0 else 360.0 - angle_deg

    return angle

In [ ]:
def ankles_too_high(left_ankle_angle, right_ankle_angle):
    return left_ankle_angle > 150 or right_ankle_angle > 150

In [ ]:
def knees_too_low(left_knee_angle, right_knee_angle):
  return left_knee_angle < 50 or right_knee_angle < 50

In [ ]:
def check_conditions(keypoints):

        # Extract landmarks for left hip, knee, ankle, and foot index
        left_hip = keypoints[mp_pose.PoseLandmark.LEFT_HIP.value]
        left_knee = keypoints[mp_pose.PoseLandmark.LEFT_KNEE.value]
        left_ankle = keypoints[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        left_foot_index = keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
        left_heel = keypoints[mp_pose.PoseLandmark.LEFT_HEEL.value]

        # Extract landmarks for right hip, knee, ankle, and foot index
        right_hip = keypoints[mp_pose.PoseLandmark.RIGHT_HIP.value]
        right_knee = keypoints[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        right_ankle = keypoints[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        right_foot_index = keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
        right_heel = keypoints[mp_pose.PoseLandmark.RIGHT_HEEL.value]


        # Calculate the left ankle angle
        left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot_index)

        # Calculate the right ankle angle
        right_ankle_angle = calculate_angle(right_knee, right_ankle, right_foot_index)

        # Calculate the left knee angle
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

        # Calculate the right knee angle
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

        # Calculate the left foot angle
        #left_foot_angle = calculate_angle(left_ankle, left_foot_index, left_heel)

        # Calculate the right foot angle
        #left_foot_angle = calculate_angle(left_ankle, left_foot_index, left_heel)

        # Check conditions for ankle and knee angles
        ankle_condition = ankles_too_high(left_ankle_angle, right_ankle_angle)
        knee_condition = knees_too_low(left_knee_angle, right_knee_angle)

        if ankle_condition and knee_condition:
            return "Ankle too high and Knee too low"
        elif ankle_condition:
            return "Ankle too high"
        elif knee_condition:
            return "Knee too low"

        return None

In [ ]:
def calculate_toe_angle(a, b, c):
    x1, y1, z1 = a[0], a[1], a[2]
    x2, y2, z2 = b[0], b[1], b[2]
    x3, y3, z3 = c[0], c[1], c[2]

    # Calculate vectors AB and BC
    vec_ab = np.array([x2 - x1, y2 - y1, z2 - z1])
    vec_bc = np.array([x3 - x2, y3 - y2, z3 - z2])

    # Calculate the dot product of AB and BC
    dot_product = np.dot(vec_ab, vec_bc)

    # Calculate the magnitudes of vectors AB and BC
    mag_ab = np.linalg.norm(vec_ab)
    mag_bc = np.linalg.norm(vec_bc)

    # Calculate the angle in radians
    radians = np.arccos(dot_product / (mag_ab * mag_bc))

    # Convert radians to degrees
    angle_deg = np.degrees(radians)

    return angle_deg

In [ ]:
def toe_angle(keypoints):
    left_ankle = keypoints[mp_pose.PoseLandmark.LEFT_ANKLE.value]
    left_foot_index = keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
    left_heel = keypoints[mp_pose.PoseLandmark.LEFT_HEEL.value]

    right_ankle = keypoints[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_foot_index = keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
    right_heel = keypoints[mp_pose.PoseLandmark.RIGHT_HEEL.value]

    # Calculate the left foot angle
    left_foot_angle = calculate_toe_angle(left_ankle, left_foot_index, left_heel)

    # Calculate the right foot angle
    right_foot_angle = calculate_toe_angle(right_ankle, right_foot_index, right_heel)

    foot_outward_threshold = 175

    # Initialize the result string
    result_str = ""

    # Check and add left foot angle to the result string if it exceeds the threshold
    if left_foot_angle > foot_outward_threshold:
        left_foot_angle -= 175
        if left_foot_angle < 0:  # Ensure angle is positive
            left_foot_angle += 180
        result_str += f"Left Foot Outward {left_foot_angle:.2f} deg\n"

    # Check and add right foot angle to the result string if it exceeds the threshold
    if right_foot_angle > foot_outward_threshold:
        right_foot_angle -= 175
        if right_foot_angle < 0:  # Ensure angle is positive
            right_foot_angle += 180
        result_str += f"Right Foot Outward {right_foot_angle:.2f} deg\n"

    # Return the result string, or None if no angles exceeded the threshold
    return result_str

In [ ]:
# Global variable to track previous keypoint positions
prev_keypoints = None

def check_travel_distance(keypoints, threshold=0.03):
    global prev_keypoints

    if prev_keypoints is not None:
        # Extract X coordinates
        left_foot_x = keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value][0]
        right_foot_x = keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value][0]

        # Check if keypoints are valid (not None)
        if left_foot_x is not None and right_foot_x is not None:
            # Calculate the absolute difference in X coordinates between current and previous frames
            foot_displacement = abs(left_foot_x - prev_keypoints[0]) + abs(right_foot_x - prev_keypoints[1])

            # Check if the displacement exceeds the threshold
            if foot_displacement > threshold:
                direction = "Forward" if left_foot_x - prev_keypoints[0] > 0 else "Backward"
                return f"Traveling {direction}"

    # Update previous keypoints for the next iteration
    if keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value][0] is not None and keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value][0] is not None:
        prev_keypoints = (keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value][0], keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value][0])
    return None

In [ ]:
def speed_angle(a, b):
    # Calculate the absolute difference in X coordinates
    x_diff = abs(a[0] - b[0])
    # Calculate the absolute difference in Y coordinates
    y_diff = abs(a[1] - b[1])

    # Avoid division by zero if X difference is zero
    if x_diff == 0:
        return 90  # Return 90 degrees if X difference is zero

    angle_rad = abs(y_diff / x_diff)
    angle_deg = round(angle_rad * 180.0 / 3.1416, 2)  # Convert radians to degrees
    return angle_deg

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
angle_count = 0
start_frame = 0
frames_to_print = 0

def check_skipping_speed(keypoints):
    global angle_count, start_frame, frames_to_print, speed_text

    # Calculate the angle between hand and toe keypoints
    left_hand = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_hand = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_toe = keypoints[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
    right_toe = keypoints[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]

    angle_left = speed_angle(left_hand, left_toe)
    angle_right = speed_angle(right_hand, right_toe)

    # Check if the angle is greater than 90 degrees
    if angle_left > 90 or angle_right > 90:
      angle_count += 1

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    elapsed_frames = current_frame - start_frame

    # Check if 1 second has elapsed (based on FPS)
    if elapsed_frames >= fps:
        if angle_count >= 30:
            speed_text = "Skipping speed is too fast"
            frames_to_print = int(fps * 1)
        elif angle_count <= 15:
            speed_text = "Skipping speed is too slow"
            frames_to_print = int(fps * 1)
        else:
            speed_text = "Skipping speed is normal"
            frames_to_print = int(fps * 1)

        #print(f"Angle Count for this second: {angle_count}")
        #print(speed_text)

        angle_count = 0  # Reset angle count for the next second
        start_frame = current_frame  # Reset the start frame for the next second

    if frames_to_print > 0:
      frames_to_print -= 1
      return speed_text

    return None

In [ ]:
all_keypoints_detected_flag = False

# Iterate over frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame and process for keypoints
    frame = cv2.resize(frame, (output_width, output_height))
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    keypoints = extract_keypoints(frame)

    if keypoints:
        keypoints_detected = all(keypoints[idx] is not None for idx in specific_keypoints_indices)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

        if keypoints_detected:
            all_keypoints_detected_flag = True
            print("Key Points Detected")

            # If all keypoints are detected, pass the frame to the model for prediction
            result = prediction(frame)  # Pass the original frame for prediction, not the image
            print("Predicted class:", result)
            if result == "Skipping":

              #angle_count, frame_count = calculate_and_count_angles(results, angle_count, frame_count, fps)

              speed_text = check_skipping_speed(keypoints)
              if speed_text:
                print(speed_text)
                cv2.putText(image, speed_text, (50, 550), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), 2)

              toe_text = toe_angle(keypoints)
              if toe_text:
                print(toe_text)
                lines = toe_text.strip().split('\n')
                y_offset = 350
                for line in lines:
                  cv2.putText(image, line, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), 2)
                  y_offset += 100

              travel_text = check_travel_distance(keypoints)
              if travel_text:
                print(travel_text)
                cv2.putText(image, travel_text, (50, 250), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), 2)

              condition_text = check_conditions(keypoints)
              if condition_text:
                print(condition_text)
                cv2.putText(image, condition_text, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), 2)

            write_text(image, result)

        out.write(image)

    else:
        # Draw landmarks on the frame without keypoints
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

        write_text(image, 'Keypoints Not Detected')

        out.write(image)

# Release resources
cap.release()
out.release()
print("Done processing video")

# Release MediaPipe Pose model
pose.close()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 241ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Key Points Detected
1/1 [==============================] - 0s 230ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Key Points Detected
1/1 [==============================] - 0s 140ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Key Points Detected
1/1 [==============================] - 0s 136ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Key Points Detected
1/1 [==============================] - 0s 142ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Key Points Detected
1/1 [==============================] - 0s 139ms/step
Predicted class: Skipping
Skipping speed is too fast
Traveling Forward
Ankle too high
Key Points Detected
1/1 [==============================] - 0s 135ms/step
Predicted class: 